In [1]:
import pandas as pd
import numpy as np
import shutil
import os
import datetime
import glob

elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\CSIRO_GBR_ElementMapper.csv'
#Names from element mapper
regID = 'RegID'
riverCol = 'Major River'
elementCol = 'Network Element'

#Use this to write out (or not) individual outlet files
WriteIndividual = False

#Standard eReefs baseline/pre-dev
simMapperCSV = r'E:\RC10PesticideProjects\eReefs_Pesticides_ResultsMapper.csv'


#Names from sim mapper, use RegID from above
basinFolderName = 'BasinFolder'
newSimFolderName = 'SIMCODE'
#Switch these for TS Dependent or EMCMonthly
#existSimFolderName = 'TSDependentSubDir'
existSimFolderName = 'EMCMonthly2021SubDir'

# #Switch these for parent location of TS Dependent or EMCMonthly results
# #TSDependent
# parentResultsDir = r'E:\GBRFScenarios\Results'
# addRegIdDIR = True
# outPath = r'E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet'
#Empirical
parentResultsDir = r'E:\RC10PesticideProjects\results'
addRegIdDIR = False
outPath = r'E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021'

riverOutlets = 'RiverOutlets'
regAgg = 'RegionalAggregation'

#, 'Sediment - Coarse'

# #Constituents = ['Sediment - Fine', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']
Constituents = ['Ametryn', 'S-metolachlor', 'Atrazine', 'Diuron', 'Hexazinone', 'Tebuthiuron', '24-D'
                , 'Paraquat', 'Glyphosate', 'Chlorsulfuron', 'Diquat', 'Fluroxypyr', 'Haloxyfop', 'Imazapic'
               , 'Imazethapyr', 'Isoxaflutole', 'MCPA', 'Metribuzin', 'Metsulfuron-methyl', 'Pendimethalin', 'Simazine'
               , 'Terbuthylazine', 'Trifluralin', 'Aciflurofen', 'Chlorpyrifos', 'Fipronil', 'Imidacloprid', 'Prometryn', 'Triclopyr']

regionIDs = {'BM':'Burnett Mary Rebuild 2014','BU':'Burdekin Rebuild 2014','FI':'FI RC2019',
              'MW':'MW_RC10', 'WT':'WT_RC10'}





In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
if not os.path.exists(outPath + "\\" + riverOutlets):
    os.makedirs(outPath + "\\" + riverOutlets)
    print("Made dir: " + outPath + "\\" + riverOutlets)


In [4]:
if not os.path.exists(outPath + "\\" + regAgg):
    os.makedirs(outPath + "\\" + regAgg)
    print("Made dir: " + outPath + "\\" + regAgg)


In [5]:
# RiverElements = {}
# elementsTable = pd.read_csv(elementMapperCSV)

# for index, row in elementsTable.iterrows():
#     if not row[riverCol] in RiverElements.keys():
#         RiverElements[row[riverCol]] = []
    
#     RiverElements[row[riverCol]].append(row[elementCol])


# #print(RiverElements)

In [6]:
allElementsTable = pd.read_csv(elementMapperCSV)
simDetailsDF = pd.read_csv(simMapperCSV)

In [7]:
def getResultsPathInfoOutlets(regionIDString, scenarioString):
    tsPath = ''
    if addRegIdDIR:
        tsPath = parentResultsDir + '\\' + regionIDString + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    else:
        tsPath = parentResultsDir + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    return tsPath

In [8]:
def getResultsPathInfoRegAgg(regionIDString, scenarioString):
    tsPath = ''
    if addRegIdDIR:
        tsPath = parentResultsDir + '\\' + regionIDString + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\Regional_TimeSeries\\Aggregated'
    else:
        tsPath = parentResultsDir + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\Regional_TimeSeries\\Aggregated'
    return tsPath

In [9]:
def doThisSimOutlet(thisRegID, thisBasinFolderName, thisNewSimFolderName, thisExistSimFolderName, theseRiverElements):
    
    #Going to need to set up folders
    #basinOutputStr = outPath + '\\' + thisBasinFolderName
    
    outletPath = outPath + "\\" + riverOutlets
    
    basinOutputStr = os.path.join(outletPath, thisBasinFolderName)
    if not os.path.exists(basinOutputStr):
        os.makedirs(basinOutputStr)
        print("Made dir: " + basinOutputStr)
    
    
    simOutputStr = os.path.join(basinOutputStr, thisNewSimFolderName)
    if not os.path.exists(simOutputStr):
        os.makedirs(simOutputStr)
        print("Made dir: " + simOutputStr)    
    
    #Set up the totals first
    RiverElements = {}
    # elementsTable = pd.read_csv(elementMapperCSV)

    for index, row in theseRiverElements.iterrows():
         if not row[riverCol] in RiverElements.keys():
            RiverElements[row[riverCol]] = []
    
         RiverElements[row[riverCol]].append(row[elementCol])
    
    riverTotals = {}
    for theRiver in RiverElements.keys():
        riverTotals[theRiver] = pd.DataFrame()

    #Now data
    mainPath = getResultsPathInfoOutlets(thisRegID, thisExistSimFolderName)

    theFlowsDIR = os.path.join(mainPath,'Flows')
    for theRiver in RiverElements.keys():
        #We'll want flows first
        eleCount = 0
        flowFrame = pd.DataFrame()
        outputFields = ['Date']
        for theElement in RiverElements[theRiver]:
            #This will stope Outlet Node1 matching Outle Node10, Outlet Node11
            theElementUnder = theElement + '_'
            for fname in os.listdir(theFlowsDIR):
                if theElementUnder in fname:
                    #print("For this river: " + theRiver + " we have this file: " + fname)
                    flowFilePath = os.path.join(theFlowsDIR,fname)
                    flowData = pd.read_csv(flowFilePath)
                    flowData.columns = ['Date', theElement]
                    #riverTotals[theRiver].columns.drop(theElement)
                    outputFields.append(theElement)
                    if eleCount == 0:
                        #Get the structure for our totals first
                        riverTotals[theRiver] = flowData
                        
                        #Dropping this means we lose reference to the first (or only) named Element.
                        #riverTotals[theRiver].drop(theElement, axis=1, inplace=True)
                        #Now process stuff
                        flowFrame = flowData
                        #flowFrame.columns = ['Date', theElement]
                        #flowFrame['Date'] = flowData['Date']
                        #flowFrame[theElement] = flowData.iloc[:1].values#This sould give the 2nd column
                    else:
                        #Join...??
                        #flowFrame[theElement] = flowData.iloc[:1].values
                        #flowData will have flow col named after element, so won't be duplicated
                        flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
                
                    eleCount += 1
        #Totals
        #The .iloc[:, 1:] should exclude our first col, Date, from the sum
        #flowFrame['Flow_cumecs'] = flowFrame.iloc[:, 1:].sum(axis=1)
        flowFrame['Flow_cumecs'] = flowFrame.sum(axis=1)
        #Write out flow
        theOutFlowFile = os.path.join(simOutputStr, "Flow " + theRiver + " cubicmetrespersecond.csv")
        if WriteIndividual:
            flowFrame.to_csv(theOutFlowFile, index=False)
        
        #Need to join flow Total to RiverTotals
        #This should be using ONLY 'Date' and 'Flow_cumecs', thereby dropping theElement from riverTotals... but is it???
        #riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])
        #When doing Flow, we're creating these, so we should never hit the ELSE here, and vice versa foWQ
        if len(riverTotals[theRiver].columns):
            #print("We were empty doing: " + theRiver)
            riverTotals[theRiver] = flowFrame[['Date', 'Flow_cumecs']]
        else:
            #print("NOOOOOOTTTTT empty doing: " + theRiver)
            riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])

    #Now water quality
    for theWQBit in Constituents:
        wqPath = os.path.join(mainPath, theWQBit)
        if not os.path.exists(wqPath):
            continue
    
        for theRiver in RiverElements.keys():
            #We'll want flows first
            eleCount = 0
            wqFrame = pd.DataFrame()
            outputFields = ['Date']
            for theElement in RiverElements[theRiver]:
                theElementUnder = theElement + '_'
                for fname in os.listdir(wqPath):
                    if theElementUnder in fname:
                        #print("For this river: " + theRiver + " we have this file: " + fname)
                        wqFilePath = os.path.join(wqPath,fname)
                        wqData = pd.read_csv(wqFilePath)
                        wqData.columns = ['Date', theElement]
                        outputFields.append(theElement)
                        if eleCount == 0:
                            wqFrame = wqData
                        else:
                            wqFrame = pd.merge(wqFrame, wqData, how='left', left_on=['Date'], right_on=['Date'])
                
                        eleCount += 1
            #Totals
            #wqFrame[theRiver + '_kg'] = wqFrame.iloc[:, 1:].sum(axis=1)
            wqFrame[theRiver + '_kg'] = wqFrame.sum(axis=1)
            #Write out flow
            theOutFlowFile = os.path.join(simOutputStr, theWQBit + " " + theRiver + " kilograms.csv")
            if WriteIndividual:
                wqFrame.to_csv(theOutFlowFile, index=False)
            
            #Join to totals
            #rename constituent total first
            #print('theRiver: ' + theRiver)
            #print('theWQBit: ' + theWQBit)
            wqFrame.rename(columns={theRiver + '_kg':theWQBit + '_kg'}, inplace=True)
            riverTotals[theRiver] = pd.merge(riverTotals[theRiver], wqFrame[['Date', theWQBit + '_kg']], how='left', left_on=['Date'], right_on=['Date'])


    #Now write totals files
    for theRiver in RiverElements.keys():
        theOutTotalsFile = os.path.join(simOutputStr, theRiver + " Totals.csv")
        riverTotals[theRiver].to_csv(theOutTotalsFile, index=False)


    print("Done " + thisNewSimFolderName + " at " + str(datetime.datetime.now()))

In [10]:
def doThisSimRegAgg(thisRegID, thisBasinFolderName, thisNewSimFolderName, thisExistSimFolderName):
    ##fvggg
    
    #Going to need to set up folders
    #basinOutputStr = outPath + '\\' + thisBasinFolderName
    regAggPath = outPath + "\\" + regAgg
    simOutputStr = os.path.join(regAggPath, thisNewSimFolderName)
    if not os.path.exists(simOutputStr):
        os.makedirs(simOutputStr)
        print("Made dir: " + simOutputStr)
    
    #Now data
    mainPath = getResultsPathInfoRegAgg(thisRegID, thisExistSimFolderName)
    
    #copy
    for name in glob.glob(mainPath + '/*.csv'):
        shutil.copy(name, simOutputStr)
    
    

In [11]:
#This is where we will loop through sims
for index, row in simDetailsDF.iterrows():
    if row[regID] in regionIDs.keys():
        print("Doing this simulation: " + row[newSimFolderName] + " at " + str(datetime.datetime.now()))
        #Filter out the river elements for this region
        filteredElements = allElementsTable.loc[allElementsTable[regID] == row[regID]]
        doThisSimOutlet(row[regID], row[basinFolderName], row[newSimFolderName], row[existSimFolderName], filteredElements)
        doThisSimRegAgg(row[regID], row[basinFolderName], row[newSimFolderName], row[existSimFolderName])

print("All finished at " + str(datetime.datetime.now()))

Doing this simulation: BUBASE at 2021-12-23 14:28:52.657397
Done BUBASE at 2021-12-23 14:29:05.493457
Doing this simulation: FIBASE at 2021-12-23 14:29:06.279836
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RiverOutlets\Fitzroy
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RiverOutlets\Fitzroy\FIBASE
Done FIBASE at 2021-12-23 14:29:25.156389
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RegionalAggregation\FIBASE
Doing this simulation: WTBASE at 2021-12-23 14:29:25.889246
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RiverOutlets\WetTropics
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RiverOutlets\WetTropics\WTBASE
Done WTBASE at 2021-12-23 14:29:45.322243
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RegionalAggregation\WTBASE
Doing this simulation: BMBASE at 2021-12-23 14:29:46.709665
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov2021\RiverOutlets\BurnettMary
Made dir: E:\RC10PesticideProjects\CSIRO_Outputs_Nov